In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/consolidated/df_consolidated.feather
/kaggle/input/calendar/calendar.csv


In [2]:
%pip install feature_engine
%pip install sktime
%pip install darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 1.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 10.6 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.9/451.9 kB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pickle

# dataframe packages
import pandas as pd
import numpy as np

# visualization packages
import seaborn as sns
import matplotlib.pyplot as plt

# sklearn packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

# sklearn metrics
from sklearn.metrics import (   r2_score, 
                                mean_absolute_error, 
                                mean_absolute_percentage_error, 
                                mean_squared_error)

#pipeline & Custom pipeline
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

#feature engine packages
from feature_engine.creation import CyclicalFeatures

#statsmodels packages
from statsmodels.tsa.api import SimpleExpSmoothing

import warnings
warnings.filterwarnings('ignore')

import datetime
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [4]:
df = pd.read_feather("../input/consolidated/df_consolidated.feather")
calendar = pd.read_csv("../input/calendar/calendar.csv")

## 2.Calendar Features

In [5]:
class get_calendar_features(BaseEstimator, TransformerMixin):
    
    def get_quarter_feature(self , month):
        if month ==1 or month ==2 or month ==3:
            return "Qtr1"
        elif month ==4 or month ==5 or month ==6:
            return "Qtr2"
        elif month ==7 or month ==8 or month ==9:
            return "Qtr3"
        elif month ==10 or month ==11 or month ==12:
            return "Qtr4"
        
    def get_quarter_start(self, week):
        if week == 1 or week == 14 or week == 27 or week == 40:
            return 1
        else:
            return 0
        
    def get_quarter_end(self, week):
        if week == 13 or week == 26 or week == 39 or week ==52 or week == 53:
            return 1
        else:
            return 0
        
    def get_month_start(self, week):
        if week ==5 or week ==9 or week ==13 or week ==17 or week ==21 or week ==25 or week ==29 or week ==33 or week ==37 or week ==41 or week ==45 or week ==49:
            return 1
        else:
            return 0

    def fit(self, X, y = None):
        #=== Just to accomadate sklearn fit and transform method =====#
        return self

    def transform(self, X):
        X = X.copy()
        X["Qtr"] = X["month"].apply(lambda x : self.get_quarter_feature(x))
        X["Qtr_Start_Flag"] = X["week"].apply(lambda x : self.get_quarter_start(x))
        X["Qtr_End_Flag"] = X["week"].apply(lambda x : self.get_quarter_end(x))
        X["Month_Start_Flag"] = X["month"].apply(lambda x: self.get_month_start(x))
        return X

In [6]:
df["week"] = df["date"].dt.week
df = get_calendar_features().fit_transform(df)

## 3. Mean Features

In [7]:
# Total Average Sales by: item, state, store, cat and dept
df['iteam_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)

# Sales average by 
df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)

In [8]:
#===== Data Preprocessing ========#
df = df.sort_values(by = ["id","date"], ascending = True)
df["date"] = df["date"].astype("datetime64[ns]")
print("Shape of train: ", df.shape)
calendar_subset = calendar[["date","event_name_1","event_type_1","event_name_2","event_type_2"]].drop_duplicates().fillna("No_Event")
calendar_subset["date"] = calendar_subset["date"].astype("datetime64[ns]")
calendar_subset

Shape of train:  (7891614, 43)


,date,event_name_1,event_type_1,event_name_2,event_type_2
0,2011-01-29,No_Event,No_Event,No_Event,No_Event
1,2011-01-30,No_Event,No_Event,No_Event,No_Event
2,2011-01-31,No_Event,No_Event,No_Event,No_Event
3,2011-02-01,No_Event,No_Event,No_Event,No_Event
4,2011-02-02,No_Event,No_Event,No_Event,No_Event
...,...,...,...,...,...
1964,2016-06-15,No_Event,No_Event,No_Event,No_Event
1965,2016-06-16,No_Event,No_Event,No_Event,No_Event
1966,2016-06-17,No_Event,No_Event,No_Event,No_Event
1967,2016-06-18,No_Event,No_Event,No_Event,No_Event


# Calendar & Special Events & Indicator Features

In [9]:
%time 
df = pd.merge(df.drop(["event_name_1","event_type_1","event_name_2","event_type_2"], axis = 1),
         calendar_subset, how = "left", on = "date")
#====== special events that are present in event_name_2 and event_type_2 they are also present in 1 so excluding those rows ===========#
df = df.drop(["event_type_2", "event_name_2"], axis = 1)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [ ]:
type2_mappings = {"2011-04-24":"Cultural","2013-05-05":"Cultural", 
                  "2014-04-20":"Religious","2014-06-15":"Cultural"}
name2_mappings = {"2011-04-24":"Easter","2013-05-05":"Cinco De Mayo", 
                  "2014-04-20":"OrthodoxEaster","2014-06-15":"Father's day"}

def cust_mapping(train, dict_map, date_col ,col_name):
    for i in dict_map.keys():
        train[col_name] = np.where(train[date_col]==i, dict_map[i], train[col_name])
        
df = df.pipe(cust_mapping, dict_map = type2_mappings, date_col = "date", col_name= "event_type_1")
df = df.pipe(cust_mapping, dict_map = name2_mappings, date_col = "date", col_name= "event_name_1")

## Special Events - Calendar

In [10]:
type1_mappings = {"No_Event":0, "Sporting":1, "Cultural":2, "National":3, "Religious":4}
name1_mappings = {'Cinco De Mayo':20,'Easter':21,"Father's day":22,'Halloween':23,"Mother's day":24,'StPatricksDay':25,
'ValentinesDay':26,'Christmas':37,'ColumbusDay':38,'IndependenceDay':39,'LaborDay':310,'MartinLutherKingDay':311,
'MemorialDay':312,'NewYear':313,'PresidentsDay':314,'Thanksgiving':315,'VeteransDay':316,'No_Event':0,
'Chanukah End':418,'Eid al-Fitr':419,'EidAlAdha':420,'LentStart':421,'LentWeek2':422,'OrthodoxChristmas':423,'OrthodoxEaster':424,
'Pesach End':425,'Purim End':426,'Ramadan starts':427,'NBAFinalsEnd':128,'NBAFinalsStart':129,'SuperBowl':130,}
#==== Label Encoding ======#
df["event_name_1"] = df["event_name_1"].map(name1_mappings)
df["event_type_1"] = df["event_type_1"].map(type1_mappings)

## Lag Features - Calendar

In [11]:
def lag_features(df, lag, col_name ):
    for i in range(1, lag+1):
        df[col_name+"_lag"+str(i)] = df[col_name].shift(i).fillna(0)      

def lead_features(df, lead, col_name ):
    for i in range(1, lead+1):
        df[col_name+"_lead"+str(i)] = df[col_name].shift(-i).fillna(0)        

lag_features(df, lag = 6, col_name = "event_type_1")
lead_features(df, lead = 6, col_name = "event_type_1")
lag_features(df, lag = 6, col_name = "event_name_1")
lead_features(df, lead = 6, col_name = "event_name_1")

## 4. Feature Creation

In [12]:
class feature_engine(BaseEstimator, TransformerMixin):

    def __init__(self, start_week:int, trend:bool, quarter:bool, mv:bool, train_test_split:bool, cyclical:bool):
        self.trend = trend
        self.quarter = quarter
        self.mv = mv
        self.train_test_split = train_test_split
        self.cyclical = cyclical
        self.start_week = start_week
        
        
    def get_cyclical_feature(self):
        cyclical = CyclicalFeatures(variables = ["weekday","wday","month"], drop_original = True )
        X_train = cyclical.fit_transform(self.X_train)
        X_test = cyclical.fit_transform(self.X_test)
        return X_train, X_test
        
    
    def get_trend_feature(self):
        model = SimpleExpSmoothing(self.y_train["sold"]).fit(smoothing_level = 0.14, optimized = False)
        self.X_train["SES_13"] = model.fittedvalues
        self.X_test["SES_13"] = model.forecast(self.X_test.shape[0]).values
        return self.X_train, self.X_test
    
    def get_quarter_indicators(self):
        qrt_df = pd.concat([self.X_train, self.X_test],axis = 0)
        #==Qtr_Start_lag, Qtr_Start_Lead
        qrt_df["Lag_Qtr_Start"] = qrt_df["Qtr_Start_Flag"].shift(1).fillna(0)
        qrt_df["Lead_Qtr_Start"] = qrt_df["Qtr_Start_Flag"].shift(-1).fillna(0)
        #==Qtr_End_lag, Qtr_End_Lead
        qrt_df["Lag_Qtr_End"] = qrt_df["Qtr_End_Flag"].shift(1).fillna(0)
        qrt_df["Lead_Qtr_End"] = qrt_df["Qtr_End_Flag"].shift(-1).fillna(0)
        
        train = qrt_df[qrt_df.d<=self.start_week]
        test = qrt_df[qrt_df.d>self.start_week]
        
        return train, test
        
    def get_moving_average(self):
        train = pd.concat([self.X_train, self.y_train["sold"]], axis = 1).copy()
        test = pd.concat([self.X_test, self.y_test["sold"]], axis = 1).copy()
        
        # Weekly average and monthly average for the train dataset
        train["weekly_average"] = train.groupby(["week"])["sold"].transform("mean")
        train["monthly_average"] = train.groupby(["month"])["sold"].transform("mean")
        
        # test dataset averages
        weekly_average = train.groupby(["week"])["sold"].mean().reset_index()
        monthly_average = train.groupby(["month"])["sold"].mean().reset_index()
        
        # Merge Test with Weekly_Avg and Monthly_Avg
        def merge(df1, df2, col, col_name):
            
            df2 = df2.rename(columns = {"sold":col_name})
            
            df1 = pd.merge(df1, df2, how = "left", on = None, left_on = col, right_on = col,
                           left_index= False, right_index = False, sort = True, copy = True, indicator = False)
            
            return df1
        
        test = merge(df1 = test, df2 = weekly_average, col = "week", col_name = "weekly_average")
        test = merge(df1 = test, df2 = monthly_average, col = "month", col_name = "monthly_average")
        
        train = train.drop("sold", axis = 1)
        test = test.drop("sold", axis = 1)
        
        return train, test
    
    def get_in_outseason_indicator(self):
        train = pd.concat([self.X_train, self.y_train["sold"]], axis = 1).copy()
        self._season_dataframe_ = train.copy()
        #============== Year with full Weeks ==========#
        year_week_corr = self._season_dataframe_.groupby("year")["week"].count().reset_index()
        year_week_corr["Flag"] = year_week_corr["week"].apply(lambda x : 1 if x==365 or x==366 else 0)
        ytc = list(year_week_corr[year_week_corr["Flag"]==1]["year"].unique())
        if len(ytc)!=0:
            #=========== Calculated cummulative contribution percentage to check =======#
            self._season_dataframe_ = self._season_dataframe_[self._season_dataframe_["year"].isin(ytc)].\
                                groupby("Qtr").agg({"sold":"mean"})
            self._season_dataframe_["Contr(%)"] = self._season_dataframe_["sold"]/self._season_dataframe_["sold"].sum()
            self._season_dataframe_ = self._season_dataframe_.sort_values("Contr(%)", ascending = False).reset_index()
            self._season_dataframe_["Cum_Contr(%)"] = self._season_dataframe_["Contr(%)"].cumsum()
            #============ Cum_Contr(%) <=.80 "InSeason = 1 & >0.80", OutSeason = 0" =======#
            self._season_dataframe_["IN_OUT_SEASON"] = np.where(self._season_dataframe_["Cum_Contr(%)"]<=0.80, 1, 0)
            return self._season_dataframe_[["Qtr","IN_OUT_SEASON"]].drop_duplicates()  
        else:
            self._season_dataframe_["IN_OUT_SEASON"] = 1
            return self._season_dataframe_[["Qtr","IN_OUT_SEASON"]].drop_duplicates()
        
                
    
    def get_train_test_split(self):
        df = self.X.copy()
        
        X_train = df[df['d']<=self.start_week].drop("sold", axis = 1).\
                    sort_values(by = ["id","d"], ascending = True).copy()

        X_test = df[df['d']>self.start_week].drop("sold", axis = 1).\
                    sort_values(by = ["id","d"], ascending = True).copy()

        y_train = df[df["d"]<=self.start_week][["id","d","sold"]].\
                    sort_values(by = ["id","d"], ascending = True).copy()

        y_test = df[df["d"]>self.start_week][["id","d","sold"]].\
                    sort_values(by = ["id","d"], ascending = True).copy()
        
        return X_train, X_test, y_train, y_test
    
    def get_train_validation_split(self):
        
        df = pd.merge(self.X_train, self.y_train, how = "inner", on = ["id","d"])

        X_train_val = df[df['d']<=1899].drop("sold", axis = 1).\
                    sort_index(ascending = True).copy()

        X_val = df[(df['d']>1899) & (df['d']<=self.start_week)].drop("sold", axis = 1).\
                    sort_index(ascending = True).copy()

        y_train_val = df[df["d"]<=1899][["id","d","sold"]].\
                    sort_index(ascending = True).copy()

        y_val = df[(df['d']>1899) & (df['d']<=self.start_week)][["id","d","sold"]].\
                    sort_index(ascending = True).copy()
        
        return X_train_val, X_val, y_train_val, y_val
        
        
    def fit(self, X, y= None):
        #Just to accomdate sklearn fit and transform package#
        return self
        
    def transform(self, X):
        self.X = X.copy()
        #===== train_test_split =======#
        self.X_train, self.X_test, self.y_train, self.y_test = self.get_train_test_split()
        
        #===== trend feature =======#
        self.X_train, self.X_test = self.get_trend_feature()
        
        #===== quarter feature =====#
        self.X_train, self.X_test = self.get_quarter_indicators()
        
        #===== moving average ======#
        self.X_train, self.X_test = self.get_moving_average()
        
        #==== in&Out Season Indicator =====#
        self.season_indicator = self.get_in_outseason_indicator()
        self.X_train = pd.merge(self.X_train, self.season_indicator, how = "left", on = "Qtr")
        self.X_test = pd.merge(self.X_test, self.season_indicator, how = "left", on = "Qtr")
        
        #===== Cyclical feature ========#
        self.X_train, self.X_test = self.get_cyclical_feature()
        
        
        return self.X_train, self.X_test, self.y_train, self.y_test

In [13]:
X_train_df = pd.DataFrame([])
X_test_df = pd.DataFrame([])
y_train_df = pd.DataFrame([])
y_test_df = pd.DataFrame([])
for i in df["id"].unique():
    X_train, X_test, y_train, y_test = feature_engine(start_week = 1913, trend = True, quarter = True, mv = True, train_test_split = True, cyclical = True).fit_transform(df[df["id"]==i])
    X_train_df = pd.concat([X_train_df, X_train])
    X_test_df = pd.concat([X_test_df, X_test])
    y_train_df = pd.concat([y_train_df, y_train])
    y_test_df = pd.concat([y_test_df, y_test])
    print('Combination ',int(list(df["id"].unique()).index(i)+1), "of", len(df["id"].unique()))

Combination  1 of 4170
Combination  2 of 4170
Combination  3 of 4170
Combination  4 of 4170
Combination  5 of 4170
Combination  6 of 4170
Combination  7 of 4170
Combination  8 of 4170
Combination  9 of 4170
Combination  10 of 4170
Combination  11 of 4170
Combination  12 of 4170
Combination  13 of 4170
Combination  14 of 4170
Combination  15 of 4170
Combination  16 of 4170
Combination  17 of 4170
Combination  18 of 4170
Combination  19 of 4170
Combination  20 of 4170
Combination  21 of 4170
Combination  22 of 4170
Combination  23 of 4170
Combination  24 of 4170
Combination  25 of 4170
Combination  26 of 4170
Combination  27 of 4170
Combination  28 of 4170
Combination  29 of 4170
Combination  30 of 4170
Combination  31 of 4170
Combination  32 of 4170
Combination  33 of 4170
Combination  34 of 4170
Combination  35 of 4170
Combination  36 of 4170
Combination  37 of 4170
Combination  38 of 4170
Combination  39 of 4170
Combination  40 of 4170
Combination  41 of 4170
Combination  42 of 4170
C

In [16]:
X_train_df.reset_index(drop= True).to_feather("/kaggle/working/X_train.feather")
y_train_df.reset_index(drop= True).to_feather("/kaggle/working/y_train.feather")
X_test_df.reset_index(drop= True).to_feather("/kaggle/working/X_test.feather")
y_test_df.reset_index(drop= True).to_feather("/kaggle/working/y_test.feather")